In [33]:
import pandas as pd
import numpy as np
import os
import shutil

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from keras.callbacks.callbacks import ModelCheckpoint

REPLACE CELL BELOW WITH GETTING REAL DATA

In [25]:
#split into test and train directories
source1 = "images/small2"
dest11 = "cnn_images/test"
dest12 = "cnn_images/training"
classes = {0:"/crohns", 1: "/bowel", 2:"/none"}
files = os.listdir(source1)
print(len(files))
import numpy as np

f = open('classifications.txt', 'r').read()
samples_to_label = eval(f)
print(len(samples_to_label))

for f in files:
    sample = f[:-4]
    classification = samples_to_label[sample]
    if np.random.rand(1) < 0.4:
        shutil.copyfile(source1 + '/'+ f, dest11 + classes[classification]+'/'+ f)
    else:
        shutil.copyfile(source1 + '/'+ f, dest12 + classes[classification]+'/'+ f)
        


322
322


In [26]:
#load images
train_data_dir = 'cnn_images/training/'
test_data_dir = 'cnn_images/test/'

train_gen = ImageDataGenerator(rescale = 1.0/255)

test_gen = ImageDataGenerator(rescale = 1.0/255)

train_generator = train_gen.flow_from_directory(train_data_dir,
                                        classes = ['crohns', 'bowel', 'none'],
                                        batch_size = 20,
                                        class_mode = "categorical",
                                        target_size = (150, 150))

test_generator = test_gen.flow_from_directory(test_data_dir,
                                        classes = ['crohns', 'bowel', 'none'],
                                        batch_size = 20,
                                        class_mode = "categorical",
                                        target_size = (150, 150))

Found 184 images belonging to 3 classes.
Found 138 images belonging to 3 classes.


In [27]:
'''
Build CNN model with multiple layers.
'''
def build_cnn(dropout_rate):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation = 'relu', input_shape=(150, 150, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering="th"))

    model.add(Conv2D(32, (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering="th"))

    model.add(Conv2D(64, (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering="th"))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(3, activation='softmax'))
    return model


In [23]:
#Define set of options for hyperparameters
dropout_rates = [0.1, 0.2,  0.5, 0.8]
learning_rates = [0.001, 0.01, 0.1]
accs = np.zeros((len(dropout_rates), len(learning_rates)))

In [28]:
for i in range(len(dropout_rates)):
    dropout_rate = dropout_rates[i]
    for j in range(len(learning_rates)):
        learning_rate = learning_rates[j]
        model = build_cnn(dropout_rate)
        #compile thee model
        model.compile(optimizer=RMSprop(lr = learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
        history = model.fit_generator(train_generator,
                                      validation_data = test_generator,
                                      steps_per_epoch = 100,
                                      epochs = 15,
                                      validation_steps = 50,
                                      verbose = 1)        
        accs[i][j] = max(history.history["val_accuracy"])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`
  # Remove the CWD from sys.path while we load stuff.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`
  del sys.path[0]


Epoch 1/15
100/100 [==============================] - 72s 725ms/step - loss: 1.4983 - accuracy: 0.6538 - val_loss: 0.9428 - val_accuracy: 0.6653
Epoch 2/15
100/100 [==============================] - 68s 681ms/step - loss: 0.9257 - accuracy: 0.6560 - val_loss: 0.8853 - val_accuracy: 0.6663
Epoch 3/15
100/100 [==============================] - 65s 647ms/step - loss: 0.8828 - accuracy: 0.6772 - val_loss: 0.8798 - val_accuracy: 0.6704
Epoch 4/15
100/100 [==============================] - 65s 650ms/step - loss: 0.9062 - accuracy: 0.6826 - val_loss: 1.1392 - val_accuracy: 0.6613
Epoch 5/15
100/100 [==============================] - 61s 607ms/step - loss: 0.8510 - accuracy: 0.6848 - val_loss: 0.5979 - val_accuracy: 0.6744
Epoch 6/15
100/100 [==============================] - 59s 593ms/step - loss: 0.8996 - accuracy: 0.6848 - val_loss: 0.9972 - val_accuracy: 0.6653
Epoch 7/15
100/100 [==============================] - 61s 610ms/step - loss: 0.8745 - accuracy: 0.6848 - val_loss: 1.1608 - val_ac

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`
  # Remove the CWD from sys.path while we load stuff.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`
  del sys.path[0]


Epoch 1/15
100/100 [==============================] - 52s 519ms/step - loss: 96.7071 - accuracy: 0.6707 - val_loss: 0.8625 - val_accuracy: 0.6663
Epoch 2/15
100/100 [==============================] - 52s 518ms/step - loss: 0.8482 - accuracy: 0.6848 - val_loss: 1.0497 - val_accuracy: 0.6673
Epoch 3/15
100/100 [==============================] - 52s 517ms/step - loss: 0.8630 - accuracy: 0.6848 - val_loss: 0.8872 - val_accuracy: 0.6633
Epoch 4/15
100/100 [==============================] - 55s 547ms/step - loss: 0.8527 - accuracy: 0.6848 - val_loss: 0.7512 - val_accuracy: 0.6663
Epoch 5/15
100/100 [==============================] - 56s 565ms/step - loss: 0.8446 - accuracy: 0.6848 - val_loss: 1.1224 - val_accuracy: 0.6673
Epoch 6/15
100/100 [==============================] - 56s 564ms/step - loss: 0.8434 - accuracy: 0.6848 - val_loss: 0.8747 - val_accuracy: 0.6694
Epoch 7/15
100/100 [==============================] - 56s 565ms/step - loss: 0.8502 - accuracy: 0.6848 - val_loss: 0.8590 - val_a

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`
  # Remove the CWD from sys.path while we load stuff.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`
  del sys.path[0]


...

Epoch 1/15
100/100 [==============================] - 52s 524ms/step - loss: 1.9921 - accuracy: 0.6380 - val_loss: 0.8961 - val_accuracy: 0.6663
Epoch 2/15
100/100 [==============================] - 58s 583ms/step - loss: 0.9543 - accuracy: 0.6810 - val_loss: 0.9614 - val_accuracy: 0.6673
Epoch 3/15
100/100 [==============================] - 57s 574ms/step - loss: 0.9261 - accuracy: 0.6804 - val_loss: 0.7125 - val_accuracy: 0.6694
Epoch 4/15
100/100 [==============================] - 53s 529ms/step - loss: 0.8962 - accuracy: 0.6842 - val_loss: 1.0632 - val_accuracy: 0.6633
Epoch 5/15
100/100 [==============================] - 53s 529ms/step - loss: 0.8968 - accuracy: 0.6848 - val_loss: 0.9564 - val_accuracy: 0.6633
Epoch 6/15
100/100 [==============================] - 55s 554ms/step - loss: 0.8915 - accuracy: 0.6848 - val_loss: 0.9850 - val_accuracy: 0.6714
Epoch 7/15
100/100 [==============================] - 52s 524ms/step - loss: 0.8588 - accuracy: 0.6842 - val_loss: 1.0601 - val_ac

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`
  # Remove the CWD from sys.path while we load stuff.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`
  del sys.path[0]


Epoch 1/15
100/100 [==============================] - 62s 617ms/step - loss: 161.0026 - accuracy: 0.6658 - val_loss: 0.7570 - val_accuracy: 0.6684
Epoch 2/15
100/100 [==============================] - 57s 574ms/step - loss: 0.8602 - accuracy: 0.6848 - val_loss: 0.9258 - val_accuracy: 0.6623
Epoch 3/15
100/100 [==============================] - 58s 583ms/step - loss: 0.8526 - accuracy: 0.6848 - val_loss: 0.9064 - val_accuracy: 0.6673
Epoch 4/15
100/100 [==============================] - 58s 578ms/step - loss: 0.8497 - accuracy: 0.6848 - val_loss: 1.3127 - val_accuracy: 0.6684
Epoch 5/15
100/100 [==============================] - 59s 588ms/step - loss: 0.8415 - accuracy: 0.6848 - val_loss: 0.9569 - val_accuracy: 0.6663
Epoch 6/15
100/100 [==============================] - 57s 567ms/step - loss: 0.8335 - accuracy: 0.6848 - val_loss: 0.9839 - val_accuracy: 0.6633
Epoch 7/15
100/100 [==============================] - 57s 572ms/step - loss: 0.8427 - accuracy: 0.6848 - val_loss: 0.6299 - val_

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`
  # Remove the CWD from sys.path while we load stuff.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`
  del sys.path[0]


Epoch 1/15
100/100 [==============================] - 55s 552ms/step - loss: 1266217.3285 - accuracy: 0.6712 - val_loss: 0.9399 - val_accuracy: 0.6653
Epoch 2/15
100/100 [==============================] - 55s 546ms/step - loss: 0.8652 - accuracy: 0.6848 - val_loss: 0.9693 - val_accuracy: 0.6694
Epoch 3/15
100/100 [==============================] - 67s 673ms/step - loss: 0.8573 - accuracy: 0.6848 - val_loss: 1.0451 - val_accuracy: 0.6653
Epoch 4/15
100/100 [==============================] - 60s 599ms/step - loss: 0.8555 - accuracy: 0.6848 - val_loss: 0.8923 - val_accuracy: 0.6623
Epoch 5/15
100/100 [==============================] - 56s 557ms/step - loss: 0.8583 - accuracy: 0.6848 - val_loss: 1.0163 - val_accuracy: 0.6724
Epoch 6/15
100/100 [==============================] - 56s 563ms/step - loss: 0.8315 - accuracy: 0.6848 - val_loss: 0.8974 - val_accuracy: 0.6623
Epoch 7/15
100/100 [==============================] - 55s 551ms/step - loss: 0.8452 - accuracy: 0.6848 - val_loss: 0.9406 - 

In [29]:
#Get the accuracy and parameters for which we have best accuracy
accuracy_df = pd.DataFrame(data=accs, index=dropout_rates, columns=learning_rates)  
accuracy_df.to_csv("CNN_all_results.csv")

ind = np.unravel_index(np.argmax(accs, axis=None), accs.shape) 
dropout_rate = dropout_rates[ind[0]]
learning_rate = learning_rates[ind[1]]

with open("CNN_best_result.txt", "a+") as f:
    f.write("Best Model Dropout Rate: %f" %dropout_rate)
    f.write("Best Model Learning Rate: %f" %learning_rate)
    f.write("Best Model Loss: %f" %accs[ind[0]][ind[1]])

In [30]:
accuracy_df

,0.001,0.010,0.100
0.1,0.674442,0.675456,0.671400
0.2,0.718053,0.670385,0.675456
0.5,0.673428,0.676471,0.670385
0.8,0.674442,0.670732,0.672414


In [36]:
filepath="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [37]:
#train and save model with best hyperparameters
model.compile(optimizer=RMSprop(lr = learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit_generator(train_generator,
                              validation_data = test_generator,
                              steps_per_epoch = 100,
                              epochs = 15,
                              validation_steps = 50,
                              callbacks = callbacks_list,
                              verbose = 1)
model.save('best_CNN_model.h5')


Epoch 1/15
100/100 [==============================] - 57s 569ms/step - loss: 0.8514 - accuracy: 0.6848 - val_loss: 0.9670 - val_accuracy: 0.6653
Epoch 2/15


/opt/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


100/100 [==============================] - 54s 542ms/step - loss: 0.8438 - accuracy: 0.6848 - val_loss: 0.8879 - val_accuracy: 0.6653
Epoch 3/15
100/100 [==============================] - 54s 544ms/step - loss: 0.8366 - accuracy: 0.6848 - val_loss: 0.8721 - val_accuracy: 0.6673
Epoch 4/15
100/100 [==============================] - 54s 545ms/step - loss: 0.8391 - accuracy: 0.6848 - val_loss: 0.6780 - val_accuracy: 0.6684
Epoch 5/15
100/100 [==============================] - 56s 556ms/step - loss: 0.8499 - accuracy: 0.6848 - val_loss: 1.0031 - val_accuracy: 0.6694
Epoch 6/15
100/100 [==============================] - 53s 535ms/step - loss: 0.8357 - accuracy: 0.6848 - val_loss: 0.9399 - val_accuracy: 0.6653
Epoch 7/15
100/100 [==============================] - 52s 523ms/step - loss: 0.8352 - accuracy: 0.6848 - val_loss: 0.6910 - val_accuracy: 0.6657
Epoch 8/15
100/100 [==============================] - 53s 525ms/step - loss: 0.8443 - accuracy: 0.6848 - val_loss: 0.8738 - val_accuracy: 0.6

/opt/anaconda3/lib/python3.7/site-packages/keras/engine/saving.py:165: UserWarning: TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file.You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).
  'TensorFlow optimizers do not '


In [1]:
classes = open('classifications.txt', 'r').read()
classes = eval(classes)
y = np.array(list(classes.values()))

NameError: name 'np' is not defined